# コンピューティングを操作する

Azure Machine Learning ジョブとしてスクリプトを実行する場合は、ジョブ実行の実行コンテキストを定義する必要があります。 主要な構成の 1 つは、スクリプトを実行するコンピューティング先です。 これは、ローカル ワークステーション (この場合はコンピューティング インスタンス)、またはリモート コンピューティング ターゲット (オンデマンドでプロビジョニングされる Azure Machine Learning マネージド コンピューティング クラスターなど) です。

このノートブックでは、コンピューティング クラスターを作成し、ジョブのコンピューティング ターゲットを確認します。

## 開始する前に

このノートブックでコードを実行するには、最新バージョンの **azureml-ai-ml** パッケージが必要です。 次のセルを実行して、パッケージがインストールされていることを確認します。

> **注**:
> **azure-ai-ml** パッケージがインストールされていない場合は、`pip install azure-ai-ml` を実行してインストールします。

In [ ]:
## ワークスペースに接続する

必要な SDK パッケージがインストールされているため、ワークスペースに接続できます。

ワークスペースに接続するには、識別子パラメーター (サブスクリプション ID、リソース グループ名、ワークスペース名) が必要です。 リソース グループ名とワークスペース名は既に入力されています。 コマンドを完了するには、サブスクリプション ID のみが必要です。

必要なパラメーターを見つけるには、スタジオの右上にあるサブスクリプションとワークスペース名をクリックします。 右側にペインが開きます。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> サブスクリプション ID をコピーし、**YOUR-SUBSCRIPTION-ID** をコピーした値に置き換えます。 </p>

## コンピューティング クラスターを作成する

多くの場合、ローカル コンピューティングリソースでは、大量のデータを処理する必要がある複雑な実験や長時間実行される実験を処理するには十分でない場合があります。また、クラウドでコンピューティング リソースを動的に作成して使用する機能を活用する場合もあります。 Azure Machine Learning は、さまざまなコンピューティング ターゲットをサポートしており、これをワークスペースで定義し、ジョブの実行に使用できます。リソースの料金は、それらを使用する場合にのみ支払います。

コンピューティング クラスターは、[Azure Machine Learning スタジオ](https://ml.azure.com)で、Python SDK または Azure CLI を使用して作成できます。 次のコード セルは、お使いのワークスペースに `aml-cluster` という名前のコンピューティング クラスターが存在するかどうかを確認し、存在しない場合は作成します。

In [ ]:
コンピューティング クラスターを作成した後は、次の構成のみを変更できます。

- `min_instances`: ノードの最小数
- `max_instances`: ノードの最大数
- `idle_time_before_scale_down`: スケールダウンする前のアイドル時間

現在、コンピューティング クラスター `aml-cluster` でスケーリングできるのは、最大 1 つのノードのみです。 並列コンピューティングを可能にするために、2 つに変更しましょう。

In [ ]:
コンピューティング クラスターが更新されると、その属性を出力して、その構成を確認できます。

## スクリプトを作成してモデルをトレーニングする

モデルをトレーニングするには、まず **src** フォルダーに **diabetes_training.py** スクリプトを作成します。 このスクリプトでは、トレーニング データと同じフォルダー内の **diabetes.csv** ファイルが使用されます。

In [ ]:
## コンピューティング クラスターでジョブを実行する

これで、作成したコンピューティング クラスターでジョブを実行する準備が整いました。

> **注**:
> コンピューティング クラスターで 0 から 1 つのノードへのスケーリングが必要になるので、ジョブの開始には時間がかかります。 コンピューティング クラスターの準備ができたら、スクリプトが実行されます。 ジョブが完了すると、コンピューティング クラスターはゼロ ノードにスケール ダウンします。 コンピューティング クラスターの状態は、 **[コンピューティング]** ページで確認できます。

After you've created a compute cluster, you can only change the configuration for:

- `min_instances`: Minimum number of nodes
- `max_instances`: Maximum number of nodes
- `idle_time_before_scale_down`: Idle time before scale down

Currently, your compute cluster `aml-cluster` can only scale do a maximum of one node. Let's change that to two, to allow for parallel compute.

In [ ]:
from azure.ai.ml.entities import AmlCompute

cluster_scale = AmlCompute(
    name="aml-cluster",
    max_instances=2,
)
ml_client.begin_create_or_update(cluster_scale)

When the compute cluster is updated, you can verify its configuration by printing its attributes.

In [ ]:
cpu_cluster = ml_client.compute.get("aml-cluster")

print (
        f"AMLCompute with name {cpu_cluster.name} has a maximum of {cpu_cluster.max_instances} nodes"
    )

## Create a script to train a model

To train a model, you'll first create the **diabetes_training.py** script in the **src** folder. The script uses the **diabetes.csv** file in the same folder as the training data.

In [ ]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

## Run a job on a compute cluster

Now, you're ready to run the job on the compute cluster you created.

> **Note**:
> The job will take some time to start as the compute cluster will need to scale from zero to one node. Once the compute cluster is ready, the script will be run. When the job has finished, the compute cluster will scale back down to zero nodes. You can review the compute cluster's status in the **Compute** page.

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-cluster",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)